In [151]:
import pandas as pd
import numpy as np

In [152]:
! git clone https://github.com/andrealodi/ORIE-5213-Spring2023.git

3037.06s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


fatal: destination path 'ORIE-5213-Spring2023' already exists and is not an empty directory.


In [153]:
initial_inven= pd.read_json("ORIE-5213-Spring2023/Bike sharing/Dataset/Initial_Inven.json")

In [154]:
initial_inven

,0
0,7
1,0
2,1
3,9
4,0
5,9
6,5
7,18
8,3
9,5


In [155]:
# number of bikes initially at stand 20
initial_inven.iloc[20]

0    13
Name: 20, dtype: int64

In [156]:
simu0= pd.read_json("ORIE-5213-Spring2023/Bike sharing/Dataset/simu0_0.json")

In [157]:
simu0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       1458 non-null   int64
 1   1       1458 non-null   int64
 2   2       1458 non-null   int64
 3   3       1458 non-null   int64
dtypes: int64(4)
memory usage: 45.7 KB


In [158]:
simu0.describe()

,0,1,2,3
count,1458.000000,1458.000000,1458.000000,1458.000000
mean,834.290123,14.154321,846.270919,13.951989
std,293.686675,9.045841,295.914786,9.105823
min,4.000000,0.000000,1.000000,0.000000
25%,553.750000,6.000000,565.250000,5.000000
50%,903.500000,15.000000,911.500000,14.000000
75%,1060.750000,22.000000,1077.000000,22.000000
max,1439.000000,29.000000,1429.000000,29.000000


In [159]:
simu0

,0,1,2,3
0,4,1,22,11
1,15,1,37,18
2,19,12,45,8
3,48,11,72,5
4,50,22,73,27
...,...,...,...,...
1453,1414,27,3,9
1454,1417,17,1428,3
1455,1432,0,14,8
1456,1439,9,21,29


In [160]:
# Adjust column 2 based on values in column 0
simu0.loc[simu0[0] > simu0[2], 2] += 1440


In [161]:
simu0

,0,1,2,3
0,4,1,22,11
1,15,1,37,18
2,19,12,45,8
3,48,11,72,5
4,50,22,73,27
...,...,...,...,...
1453,1414,27,1443,9
1454,1417,17,1428,3
1455,1432,0,1454,8
1456,1439,9,1461,29


In [162]:
import pandas as pd

# Define the time ranges
time_range = [(i, i + 15) for i in range(0, 1440, 15)]

# Create the function to assign the time range
def assign_time_range(value):
    for start, end in time_range:
        if start <= value < end:
            return f"{start}-{end}"
    return "Unknown"

# Create the 'departure_time' column
simu0['departure_time'] = simu0[0].apply(assign_time_range)
simu0['arrival_time'] = simu0[2].apply(assign_time_range)

In [163]:
simu0.groupby([1,'departure_time'])[0].count().reset_index()


,1,departure_time,0
0,0,1005-1020,5
1,0,1020-1035,6
2,0,1035-1050,3
3,0,1050-1065,3
4,0,1065-1080,2
...,...,...,...
948,29,855-870,1
949,29,915-930,1
950,29,930-945,2
951,29,960-975,3


In [164]:
df_rentals = simu0.groupby([1, 'departure_time'])[0].count().reset_index()
df_rentals = df_rentals.rename(columns={0:"rentals", 1: "station_id"})
df_rentals

,station_id,departure_time,rentals
0,0,1005-1020,5
1,0,1020-1035,6
2,0,1035-1050,3
3,0,1050-1065,3
4,0,1065-1080,2
...,...,...,...
948,29,855-870,1
949,29,915-930,1
950,29,930-945,2
951,29,960-975,3


In [165]:
df_returns = simu0.groupby([3, 'arrival_time'])[2].count().reset_index()
df_returns = df_returns.rename(columns={2:"returns", 3: "station_id"})
df_returns

,station_id,arrival_time,returns
0,0,1065-1080,1
1,0,1080-1095,1
2,0,1095-1110,1
3,0,1125-1140,1
4,0,1140-1155,2
...,...,...,...
952,29,945-960,1
953,29,960-975,3
954,29,975-990,1
955,29,990-1005,1


In [166]:
import os
import pandas as pd

folder_path = "ORIE-5213-Spring2023/Bike sharing/Dataset"
json_files = [file for file in os.listdir(folder_path) if (file.endswith(".json") & file.startswith("simu"))]

dataframes = []
for json_file in json_files:
    day = json_file.split("_")[1].split(".")[0]
    file_path = os.path.join(folder_path, json_file)
    df = pd.read_json(file_path)
    df["day"] = int(day)
    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True).sort_values(by='day')
combined_df

,0,1,2,3,day
22727,861,23,880,11,0
23012,1026,2,1054,29,0
23011,1026,1,1043,21,0
23010,1026,1,1041,21,0
23009,1025,14,1034,28,0
...,...,...,...,...,...
26896,662,29,688,26,499
26895,661,27,685,4,499
26894,661,1,667,7,499
26892,658,4,673,17,499


In [167]:
len(combined_df)

732539

In [168]:
rand_num = np.random.choice(1000, len(combined_df))


In [169]:
combined_df["rand"] = np.random.choice(1000, len(combined_df))
combined_df.loc[combined_df["rand"] >= 400, "bike_type"] = 'classic'
combined_df.loc[combined_df["rand"] < 400, "bike_type"] = 'electric'
combined_df.drop(['rand'], axis=1, inplace=True)

In [182]:
combined_df

,0,1,2,3,day,bike_type,dt,at,departure_time,arrival_time
22727,861,23,880,11,0,electric,861,880,855-870,870-885
23012,1026,2,1054,29,0,electric,1026,1054,1020-1035,1050-1065
23011,1026,1,1043,21,0,electric,1026,1043,1020-1035,1035-1050
23010,1026,1,1041,21,0,electric,1026,1041,1020-1035,1035-1050
23009,1025,14,1034,28,0,electric,1025,1034,1020-1035,1020-1035
...,...,...,...,...,...,...,...,...,...,...
26896,662,29,688,26,499,classic,719222,719248,660-675,675-690
26895,661,27,685,4,499,classic,719221,719245,660-675,675-690
26894,661,1,667,7,499,electric,719221,719227,660-675,660-675
26892,658,4,673,17,499,classic,719218,719233,645-660,660-675


In [171]:
combined_df['dt'] = ""
combined_df['at'] = ""

for index, row in combined_df.iterrows():
    if row['day'] != 0:
        dt = row[0] + (1440 * row['day'])
        at = row[2] + (1440 * row['day'])
    else:
        dt = row[0]
        at = row[2]
    combined_df.at[index, 'dt'] = dt
    combined_df.at[index, 'at'] = at


In [172]:
# pd.set_option('display.max_rows', None)

In [173]:
combined_df[(combined_df['day']==0) & (combined_df[2] < combined_df[0])]

,0,1,2,3,day,bike_type,dt,at
23491,1439,9,21,29,0,classic,1439,21
23490,1432,0,14,8,0,classic,1432,14
23488,1414,27,3,9,0,electric,1414,3
23487,1412,21,1,9,0,electric,1412,1
23492,1439,16,7,17,0,classic,1439,7


In [174]:
# Adjust column 2 based on values in column 0
combined_df.loc[combined_df[0] > combined_df[2], 2] += 1440


In [175]:
combined_df[(combined_df['day']==0) & (combined_df[2] < combined_df[0])]

,0,1,2,3,day,bike_type,dt,at


In [176]:
import pandas as pd

# Define the time ranges
time_range = [(i, i + 15) for i in range(0, 1440, 15)]

# Create the function to assign the time range
def assign_time_range(value):
    for start, end in time_range:
        if start <= value < end:
            return f"{start}-{end}"
    return "Unknown"

# Create the 'departure_time' column
combined_df['departure_time'] = combined_df[0].apply(assign_time_range)
combined_df['arrival_time'] = combined_df[2].apply(assign_time_range)

In [177]:
combined_df[combined_df['day']==0]

,0,1,2,3,day,bike_type,dt,at,departure_time,arrival_time
22727,861,23,880,11,0,electric,861,880,855-870,870-885
23012,1026,2,1054,29,0,electric,1026,1054,1020-1035,1050-1065
23011,1026,1,1043,21,0,electric,1026,1043,1020-1035,1035-1050
23010,1026,1,1041,21,0,electric,1026,1041,1020-1035,1035-1050
23009,1025,14,1034,28,0,electric,1025,1034,1020-1035,1020-1035
...,...,...,...,...,...,...,...,...,...,...
22516,668,20,675,19,0,classic,668,675,660-675,675-690
22515,668,11,696,22,0,classic,668,696,660-675,690-705
22513,667,12,692,2,0,electric,667,692,660-675,690-705
22036,15,1,37,18,0,classic,15,37,15-30,30-45


In [178]:
df_rentals = combined_df.groupby([1, 'departure_time', 'day'])[0].count().reset_index()
df_rentals = df_rentals.rename(columns={0:"rentals", 1: "station_id"})
df_rentals.head(20)

,station_id,departure_time,day,rentals
0,0,0-15,2,2
1,0,0-15,6,1
2,0,0-15,16,1
3,0,0-15,33,1
4,0,0-15,36,1
5,0,0-15,51,1
6,0,0-15,63,1
7,0,0-15,76,1
8,0,0-15,87,1
9,0,0-15,95,1


In [179]:
df_returns = combined_df.groupby([3, 'arrival_time', 'day'])[2].count().reset_index()
df_returns = df_returns.rename(columns={2:"returns", 3: "station_id"})
df_returns.head(20)

,station_id,arrival_time,day,returns
0,0,0-15,14,1
1,0,0-15,61,1
2,0,0-15,162,1
3,0,0-15,184,1
4,0,0-15,260,1
5,0,0-15,311,1
6,0,0-15,329,1
7,0,0-15,337,1
8,0,0-15,372,1
9,0,0-15,407,1


In [180]:
df_rentals

,station_id,departure_time,day,rentals
0,0,0-15,2,2
1,0,0-15,6,1
2,0,0-15,16,1
3,0,0-15,33,1
4,0,0-15,36,1
...,...,...,...,...
482695,29,990-1005,495,2
482696,29,990-1005,496,3
482697,29,990-1005,497,2
482698,29,990-1005,498,3


In [181]:
df_rentals.to_csv("rentals.csv")
df_returns.to_csv("returns.csv")